## Q1: Popular US baby names (35 marks)


---
### !!! VERY IMPORTANT. PLEASE READ !!!

* **Please DO NOT upload any data (TXT files, pdf files, .db file, etc) to GitHub for this part as the data is too large!**
    * There are some restriction and issues if you upload files that are too large in GitHub. See [here](https://docs.github.com/en/repositories/working-with-files/managing-large-files/about-large-files-on-github) to learn more
    * **You will lose marks if you have uploaded any of them - even if you delete them afterwards**
    * When uploading your work via GitHub Desktop, you can select what files to upload on the left by checking (and unchecking) the boxes corresponding to the files you want to (and do not want to) upload, as demonstrated in Workshop 5
        * I have modified the setting for your problem set repository so that by default it should not let you upload the `data/namesbystate` folder, the `data/namesbystate.zip`, or the `.db` file, and you should not see them available on the left
        * However, if for some reason any of the data files appear on the left, you should uncheck the boxes corresponding to those files  
    
---

In this question, we will use the US baby names dataset from The United States Social Security Administration to create a database and answer some questions.

1. Go to https://www.ssa.gov/oact/babynames/limits.html, read the [Background information](https://www.ssa.gov/oact/babynames/background.html),  download the state-specific data on baby names, save the zip file under the `data` folder and unzip the file in the `data` folder

> * Note:
>    * Each TXT file should be under the `data/namesbystate` folder. Please DO NOT add/delete/modify any of the files
>    * Please DO NOT upload any of the TXT files as mentioned above

2. [Database normalisation] Read the "readme" file in the dowloaded folder and have a look of some of the tables in the TXT files. Find out if the tables represented by the TXT files are in 1NF? 2NF? 3NF? Please explain your answer
3. [Manage data in database] Use the Python module `sqlite3` with SQL commands, create a database and create a table containing the records from all the TXT files. Please enforce the right constraints on each attribute, including setting the primary key

> * Note: 
>     * The code used should be as "automatic" as possible. For example, you should NOT need to write separate code to load each of the files, or separate SQL commands to insert each line of records
>     * Please DO NOT upload the database data (`.db` file) as mentioned above

4. [SQL query] Answer each part below using _one single SQL query_ via `sqlite3`:
    * (a) Find out the number of baby boys and the number of baby girls born in California (CA) in each year from 2010
        * Do you observe any trend?
    * (b) List out the most popular boy names and the most popular girl names with the corresponding counts in the US in every 10 years from 1910 to 2020
        * Here "the most popular boy/girl name" in a given year means it is the name with the highest number of boy/girl babies born in that year in the US with that name
        * Your result should have 22 names and 22 counts corresponding to 1910 boy, 1920 boy, ..., 2020 boy, 1910 girl, 1920 girl, ..., 2020 girl
        * You can hard code all the years if you want to
        * For (4b), you do not need to worry if there are two names with the same number of counts in this part - giving one of the names with the highest count is sufficient
    * (c) List out the most popular baby names and corresponding counts in each state in 2020, order the counts in descending order
        * In order words, for each state, provide the baby name(s) that has the highest number of babies born in 2020 with that name, and the corresponding number of babies born in 2020 with that name)
        
    Please make sure all query results are shown in the notebook. You SHOULD NOT make use of any SQL command / keywords that we did not cover in the course.
            
> * Note:
>     * Each part should be answered by using _one_ SQL query only
>         * A query involving subquery counts as one query 
>     * The result from the SQL query should answer the corresponding part _directly_ and should NOT require any further cleaning or effort. For example:
>         * You should NOT make use of Python functionality like loops, slicing, `Pandas` functionality, etc
>         * It should NOT involve eyeballing the answer
>             * Except for the question "Do you observe any trend?"
>         * It should NOT involve hard coding of values except those mentioned in the questions 
>             * For example, it is okay to hard code 2020 for part (c) as 2020 is mentioned in the question, but if you hard code the maximum count for part (c) you will lose marks as it should be found via SQL query
>             
>     * _If_ you cannot use one query to answer the question, you can answer with a few queries with some hard coding
>         * But of course you will lose some points
>     * Please state the assumptions that you have made when answering the questions

---

# 2.

The tables represented by the TXT files are in 3NF<b>

satisfies 1NF :
    
entity : name<br>
the table has exactly one value for every row-and-column intersection<br>
Unique column names <br>
there are no NULL values.<br>
Primary key and no duplicate rows<b>r

satisfies 2NF :
    
the primary key is (sex, year of birth ) and the non-key attribute 'character name' depend on the whole key, so there is no partial dependency.

satisfies 3NF :
    
non-key attribute 'character name' s non-transitively dependent on every key of the table. hence there is no ransitively dependency.





# 3

In [1]:
# create a database: names_state
import sqlite3

In [2]:
import os

if os.path.exists("names_state.db"):
    os.remove("names_state.db") 
con = sqlite3.connect("names_state.db") 
con.execute('PRAGMA foreign_keys = ON;');

create a table names_state

In [3]:
#merge texts as files
import os
import numpy as np 
files = os.listdir('data/namesbystate/')

In [4]:
# construct 2d_list (namesbystate)
old_namesbystate = []
for file_name in files:
    if file_name[0]!= '.': # make sure the file is not hidden file
        with open('data/namesbystate/'+file_name) as f: 
            for line in f :
                words = (line.strip()).split(",")
                words[2] = int(words[2])
                words[4] = int(words[4])#转化year + num of name 从str转为int
                old_namesbystate.append(words)

In [5]:
#check
old_namesbystate[0:10]

[['AK', 'F', 1910, 'Mary', 14],
 ['AK', 'F', 1910, 'Annie', 12],
 ['AK', 'F', 1910, 'Anna', 10],
 ['AK', 'F', 1910, 'Margaret', 8],
 ['AK', 'F', 1910, 'Helen', 7],
 ['AK', 'F', 1910, 'Elsie', 6],
 ['AK', 'F', 1910, 'Lucy', 6],
 ['AK', 'F', 1910, 'Dorothy', 5],
 ['AK', 'F', 1911, 'Mary', 12],
 ['AK', 'F', 1911, 'Margaret', 7]]

In [6]:
# create a table(names_state)
import sqlite3
con.execute('DROP TABLE IF EXISTS names_state;') 

con.execute('''CREATE TABLE names_state(
               stateCode TEXT NOT NULL,
               sex TEXT NOT NULL, 
               year TEXT NOT NULL CHECK(length(year) = 4),
               name TEXT NOT NULL,
               num_occ INTEGER CHECK(num_occ >= 5),
               PRIMARY KEY(stateCode,sex,year,name));''');

In [7]:
con.commit();

In [8]:
for one_namestate in old_namesbystate:
    con.execute("INSERT INTO names_state VALUES(?, ?, ?, ?, ? );", one_namestate);
con.commit();

# 4

In [9]:
#(a) 

In [10]:
%load_ext sql
%sql sqlite:///names_state.db

'Connected: @names_state.db'

#num of boys born in California (CA) in each year from 2010

In [11]:
import sqlite3

In [12]:
con = sqlite3.connect("names_state.db")
result_b = con.execute('''SELECT year, COUNT(*) 
                        FROM names_state
                        where stateCode = 'CA'AND sex ='M'
                        GROUP BY year
                        HAVING year>=2010''').fetchall()


#num of girls born in California (CA) in each year from 2010

In [13]:
con = sqlite3.connect("names_state.db")
result_g = con.execute('''SELECT year, COUNT(*)
                        FROM names_state
                        where stateCode = 'CA'AND sex ='F'
                        GROUP BY year
                        HAVING year >= 2010''').fetchall()

In [14]:
result_b

[('2010', 2912),
 ('2011', 2889),
 ('2012', 2912),
 ('2013', 2898),
 ('2014', 2940),
 ('2015', 2937),
 ('2016', 2908),
 ('2017', 2930),
 ('2018', 2853),
 ('2019', 2863),
 ('2020', 2793),
 ('2021', 2853)]

In [15]:
result_g

[('2010', 4097),
 ('2011', 3994),
 ('2012', 4102),
 ('2013', 3968),
 ('2014', 4016),
 ('2015', 3940),
 ('2016', 3870),
 ('2017', 3763),
 ('2018', 3697),
 ('2019', 3655),
 ('2020', 3614),
 ('2021', 3646)]

The number of babys girls born in CA from 2010 was decreasing year by year 

The number of babys boys born in CA from 2010 was stable, fluated between 2800 to 2900

In [16]:
#(b) 
# List out the most popular boy names and the most popular girl names with the 
# corresponding counts in the US in every 10 years from 1910 to 2020

In [17]:
result= con.execute(''' SELECT MIN(year)-1 AS ten_y, name,sex,MAX(num_occ),
CASE WHEN sex ='F' then '2020 girl'
     WHEN sex ='M' then '2020 boy'
ELSE 'NONE' END as title
FROM names_state a
WHERE year>2010  
GROUP BY sex
UNION
SELECT MIN(year)-1 AS ten_y, name,sex,MAX(num_occ),
CASE WHEN sex ='F' then '2010 girl'
     WHEN sex ='M' then '2010 boy'
ELSE 'NONE' END as title
FROM names_state 
WHERE year>2000 and year <=2010 
GROUP BY sex
UNION
SELECT MIN(year)-1 AS ten_y, name,sex,MAX(num_occ),
CASE WHEN sex ='F' then '2000 girl'
     WHEN sex ='M' then '2000 boy'
ELSE 'NONE' END as title
FROM names_state 
WHERE year>1990 and year <=2000
GROUP BY sex
UNION
SELECT MIN(year)-1 AS ten_y, name,sex,MAX(num_occ),
CASE WHEN sex ='F' then '1990 girl'
     WHEN sex ='M' then '1990 boy'
ELSE 'NONE' END as title
FROM names_state 
WHERE year>1980 and year <=1990 
GROUP BY sex
UNION
SELECT MIN(year)-1 AS ten_y, name,sex,MAX(num_occ),
CASE WHEN sex ='F' then '1980 girl'
     WHEN sex ='M' then '1980 boy'
ELSE 'NONE' END as title
FROM names_state 
WHERE year>1970 and year <=1980 
GROUP BY sex
UNION
SELECT MIN(year)-1 AS ten_y, name,sex,MAX(num_occ),
CASE WHEN sex ='F' then '1970 girl'
     WHEN sex ='M' then '1970 boy'
ELSE 'NONE' END as title
FROM names_state 
WHERE year>1960 and year <=1970 
GROUP BY sex
UNION
SELECT MIN(year)-1 AS ten_y, name,sex,MAX(num_occ),
CASE WHEN sex ='F' then '1960 girl'
     WHEN sex ='M' then '1960 boy'
ELSE 'NONE' END as title
FROM names_state 
WHERE year>1950 and year <=1960 
GROUP BY sex
UNION
SELECT MIN(year)-1 AS ten_y, name,sex,MAX(num_occ),
CASE WHEN sex ='F' then '1950 girl'
     WHEN sex ='M' then '1950 boy'
ELSE 'NONE' END as title
FROM names_state 
WHERE year>1940 and year <=1950 
GROUP BY sex
UNION
SELECT MIN(year)-1 AS ten_y, name,sex,MAX(num_occ),
CASE WHEN sex ='F' then '1940 girl'
     WHEN sex ='M' then '1940 boy'
ELSE 'NONE' END as title
FROM names_state 
WHERE year>1930 and year <=1940 
GROUP BY sex
UNION
SELECT MIN(year)-1 AS ten_y, name,sex,MAX(num_occ),
CASE WHEN sex ='F' then '1930 girl'
     WHEN sex ='M' then '1930 boy'
ELSE 'NONE' END as title
FROM names_state 
WHERE year>1920 and year <=1930 
GROUP BY sex
UNION
SELECT MIN(year)-1 AS ten_y, name,sex,MAX(num_occ),
CASE WHEN sex ='F' then '1920 girl'
     WHEN sex ='M' then '1920 boy'
ELSE 'NONE' END as title
FROM names_state 
WHERE year>1910 and year <=1920 
GROUP BY sex
UNION
SELECT MIN(year)-1 AS ten_y, name,sex,MAX(num_occ),
CASE WHEN sex ='F' then '1910 girl'
     WHEN sex ='M' then '1910 boy'
ELSE 'NONE' END as title
FROM names_state 
WHERE year>1900 and year <=1910 
GROUP BY sex

ORDER BY ten_y
''').fetchall()
result


[(1909, 'John', 'M', 1326, '1910 boy'),
 (1909, 'Mary', 'F', 2913, '1910 girl'),
 (1910, 'John', 'M', 7557, '1920 boy'),
 (1910, 'Mary', 'F', 8184, '1920 girl'),
 (1920, 'John', 'M', 7262, '1930 boy'),
 (1920, 'Mary', 'F', 7772, '1930 girl'),
 (1930, 'Mary', 'F', 4916, '1940 girl'),
 (1930, 'Robert', 'M', 6530, '1940 boy'),
 (1940, 'Linda', 'F', 7542, '1950 girl'),
 (1940, 'Robert', 'M', 10026, '1950 boy'),
 (1950, 'Patricia', 'F', 5678, '1960 girl'),
 (1950, 'Robert', 'M', 9219, '1960 boy'),
 (1960, 'Lisa', 'F', 5612, '1970 girl'),
 (1960, 'Michael', 'M', 9241, '1970 boy'),
 (1970, 'Jennifer', 'F', 6065, '1980 girl'),
 (1970, 'Michael', 'M', 6909, '1980 boy'),
 (1980, 'Jessica', 'F', 6846, '1990 girl'),
 (1980, 'Michael', 'M', 8247, '1990 boy'),
 (1990, 'Jessica', 'F', 6952, '2000 girl'),
 (1990, 'Michael', 'M', 7572, '2000 boy'),
 (2000, 'Daniel', 'M', 4170, '2010 boy'),
 (2000, 'Emily', 'F', 3416, '2010 girl'),
 (2010, 'Jacob', 'M', 3171, '2020 boy'),
 (2010, 'Sophia', 'F', 3644, '2

In [18]:
#(c) List out the most popular baby names and corresponding counts in each state 
# in 2020,order the counts in descending order

In [19]:
result_MP_2020 = con.execute('''WITH DATA_2020 AS (
                                SELECT *
                                FROM names_state
                                WHERE year == 2020)
                                SELECT stateCode, name, MAX(num_occ) AS MAX_numocc
                                FROM DATA_2020
                                GROUP BY stateCode 
                                ORDER BY MAX_numocc DESC''').fetchall()

result_MP_2020

[('CA', 'Noah', 2625),
 ('TX', 'Liam', 2209),
 ('FL', 'Liam', 1563),
 ('NY', 'Liam', 1358),
 ('IL', 'Noah', 710),
 ('PA', 'Noah', 694),
 ('NC', 'Liam', 687),
 ('NJ', 'Liam', 665),
 ('OH', 'Liam', 613),
 ('GA', 'Liam', 591),
 ('VA', 'Liam', 510),
 ('MI', 'Oliver', 495),
 ('MA', 'Noah', 458),
 ('AZ', 'Liam', 451),
 ('TN', 'Liam', 451),
 ('WA', 'Oliver', 430),
 ('IN', 'Oliver', 403),
 ('MD', 'Liam', 368),
 ('AL', 'William', 366),
 ('MN', 'Henry', 351),
 ('MO', 'Oliver', 345),
 ('WI', 'Oliver', 326),
 ('CO', 'Olivia', 322),
 ('SC', 'William', 279),
 ('KY', 'Liam', 274),
 ('UT', 'Oliver', 268),
 ('LA', 'Ava', 261),
 ('OR', 'Oliver', 235),
 ('OK', 'Olivia', 231),
 ('IA', 'Oliver', 213),
 ('CT', 'Noah', 212),
 ('KS', 'Olivia', 189),
 ('MS', 'James', 189),
 ('NV', 'Liam', 180),
 ('AR', 'Liam', 158),
 ('ID', 'Oliver', 142),
 ('NE', 'Olivia', 132),
 ('NM', 'Liam', 126),
 ('WV', 'Liam', 106),
 ('ME', 'Oliver', 92),
 ('RI', 'Liam', 78),
 ('NH', 'Lucas', 71),
 ('MT', 'Oliver', 68),
 ('DC', 'William